In [1]:
import os
os.sys.path.append("/home/kakarot/Workspace/InterFusion/")

In [2]:
import pickle
import os

import logging
import time
import numpy as np


In [3]:
import pandas as pd

In [4]:
%cd ../..

/home/kakarot/Workspace/InterFusion


In [5]:
!ls

LICENSE  README.md  algorithm  data  explib  requirements.txt  results


# Load Actual Dataset

In [44]:
df_ga = pd.read_csv("data/Google_Anlaytics/GA_MTS - GA_MTS.csv")
df_ga.head()

,country,h6Seq,cnt_per_country_and_6h,pageviews,bounce_rate,interactive_visit_rate,Chrome,Safari,Firefox,Other_browsers,desktop,mobile,tablet,Macintosh,Windows,Android,iOS,Linux,Other_OS
0,Afghanistan,58,1,1,1.0,1,100.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0
1,Afghanistan,85,1,2,0.0,1,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0
2,Afghanistan,91,1,3,0.0,1,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0
3,Afghanistan,154,1,1,1.0,1,0.0,0.0,0.0,100.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
4,Afghanistan,213,1,2,0.0,1,0.0,0.0,100.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0


In [45]:
# filter for India
df = df_ga[df_ga["country"]=="India"].reset_index().drop(columns=["interactive_visit_rate","index","country","Other_browsers","Other_OS"]).copy()
print(df.shape)
df.head()

(1464, 15)


,h6Seq,cnt_per_country_and_6h,pageviews,bounce_rate,Chrome,Safari,Firefox,desktop,mobile,tablet,Macintosh,Windows,Android,iOS,Linux
0,1,25,68,0.560,60.000,12.00,24.00,88.000,12.000,0.00,12.0,52.00,8.000,4.00,24.000
1,2,16,71,0.563,81.250,18.75,0.00,75.000,25.000,0.00,25.0,31.25,18.750,6.25,6.250
2,3,10,24,0.800,40.000,50.00,0.00,80.000,20.000,0.00,50.0,10.00,20.000,0.00,10.000
3,4,16,45,0.625,50.000,25.00,18.75,87.500,6.250,6.25,25.0,62.50,6.250,6.25,0.000
4,5,12,22,0.417,66.667,25.00,0.00,83.333,16.667,0.00,25.0,50.00,16.667,0.00,8.333


# Load Results 

In [46]:
# input vars  GA /home/kakarot/Workspace/InterFusion/results/stack_predict_2023-05-22_15-46-41_939
# path = "/home/kakarot/Workspace/InterFusion/results/stack_predict_2023-05-22_15-46-41_939/analysis_results/"
path = "/home/kakarot/Workspace/InterFusion/results/stack_predict_edited_new/analysis_results/"
mcmc = True
is_pretrain = False
tp_res = {}

if mcmc:
    assert is_pretrain is False
    try:
        tracker = pickle.load(open(path + 'mcmc_tracker.pkl', 'rb'))
    except:
        raise FileNotFoundError('cannot find mcmc_tracker.pkl',"  ",path)
    # preprocess tracker to get:
    # {idx_ed: best_score_mcmc (np.array(shape=[window_size, x_dim]))}
    for ed_idx in tracker:
        shape = tracker[ed_idx]['best_score'].shape
        assert len(shape) == 3
        tp_res[ed_idx] = tracker[ed_idx]['best_score'].reshape([shape[1], shape[2]])

In [47]:
results = {}

### ??? 
for p in [100, 150]:
    prefix = 'p=' + str(p) + ': '
    # segment-wise aggregation
    within_window_funcs = [lambda x: x[-1]]
    within_window_func_names = ['wd_last']
    assert len(within_window_func_names) == len(within_window_funcs)

    # ??
    def min_aggr_and_keep(x):
        temp = np.min(x, axis=0)
        return [temp for _ in range(len(x))]

    def ave_aggr_and_keep(x):
        temp = np.mean(x, axis=0)
        return [temp for _ in range(len(x))]

    def max_aggr_and_keep(x):
        temp = np.max(x, axis=0)
        return [temp for _ in range(len(x))]

    def min_aggr(x):
        return [np.min(x, axis=0)]

    def ave_aggr(x):
        return [np.mean(x, axis=0)]

    def max_aggr(x):
        return [np.max(x, axis=0)]

    within_interval_sc_funcs = [min_aggr_and_keep]
    within_interval_sc_func_names = ['itv_min_weight']

In [48]:
window_func = lambda x: x[-1]
anomlay_dim_scores = {}

for ed_idx in tp_res:
    dim_scores = window_func(tp_res[ed_idx]).reshape((-1))
    anomlay_dim_scores[ed_idx] = dim_scores


In [49]:
# Create a new column with default value of 0
df['Anomaly'] = 0

# Create a boolean mask by checking index presence
mask = df.index.isin(anomlay_dim_scores.keys())

# Assign 1 to the rows where the index is present
df.loc[mask, 'Anomaly'] = 1


In [51]:
duplicates = df['h6Seq'].duplicated()

# Get the rows with duplicates
duplicate_rows = df[duplicates]
duplicate_rows

,h6Seq,cnt_per_country_and_6h,pageviews,bounce_rate,Chrome,Safari,Firefox,desktop,mobile,tablet,Macintosh,Windows,Android,iOS,Linux,Anomaly


In [54]:
import plotly.graph_objects as go

imp_cols = [
 'cnt_per_country_and_6h',
 'pageviews',
 'bounce_rate',
 'Chrome',
 'Safari',
 'Firefox',
 'desktop',
 'mobile',
 'tablet',
 'Macintosh',
 'Windows',
 'Android',
 'iOS',
 'Linux']

for x in imp_cols:
    # Create a line chart
    fig = go.Figure()

    # Add a line trace for x
    fig.add_trace(go.Scatter(x=df['h6Seq'], y=df[x], mode='lines', name=x))

    # Filter the DataFrame for Anomaly equals 1
    anomaly_df = df[df['Anomaly'] == 1]

    # Add red points for Anomaly equals 1
    fig.add_trace(go.Scatter(x=anomaly_df['h6Seq'], y=anomaly_df[x],
                            mode='markers', name='Anomaly', marker=dict(color='red')))

    # Set plot title and axis labels
    fig.update_layout(title=f'Line Chart: h6Seq vs. {x}',
                    xaxis_title='h6Seq',
                    yaxis_title=x)

    # Show the plot
    fig.show()

